# Tutorial 4: Tareas Supervisadas.


### Cuerpo Docente

- Profesores: [Andrés Abeliuk](https://aabeliuk.github.io/), [Fabián Villena](https://villena.cl/).
- Profesor Auxiliar: Martín Paredes


### Objetivos del Tutorial

- Motivación y repaso de qué son los Word Embeddings.
- Entrenar nuestros propios `word embeddings` usando un dataset con fuentes de noticias de diversos medios. 💪
- Entrenar un modelo para predecir el medio correspondiente a cada noticia.

## **Repaso: Word Embedddings**

Partamos por decir que una red neuronal no es más que una serie de operaciones matemáticas sobre vectores con una gran cantidad de dimensiones (tensores). Por ende, si queremos entrenar un modelo necesitamos transformar el texto original a vectores numéricos.

Una de las soluciones más simples a este problema es la representación de Bag of Words (BoW). Si aplicamos este método a cada palabra de cada documento, tendremos un vector one hot encoding por cada palabra. Esto quiere decir que tendremos vectores del largo del vocabulario $V$, con un 1 en la posición asociada a la palabra representada.

Y estamos listos? Podemos entrenar redes neuronales?

La verdad es que no es así, y es que estamos ignorando un gran problema con este enfoque. 😞


### El gran problema de Bag of Words

Pensemos en estas 3 frases como documentos:

- $doc_1$: `¡Buenísima la marraqueta!`
- $doc_2$: `¡Estuvo espectacular ese pan francés!`
- $doc_3$: `!Buenísima esa pintura!`

Sabemos $doc_1$ y $doc_2$ hablan de lo mismo 🍞🍞👌 y que $doc_3$ 🎨 no tiene mucho que ver con los otros.

Supongamos que queremos ver que tan similares son ambos documentos.
Para esto, generamos un modelo `Bag of Words` sobre el documento, aplicando este método por cada palabra para luego tener la representación final.


Es decir, transformamos cada palabra a un vector one-hot y luego los sumamos por documento.

Por simplicidad, omitiremos algunas stopwords y consideramos pan frances como un solo token. Así nos quedaría el siguiente vocabulario:

$$v = \{buenísima, marraqueta, estuvo, espectacular, pan\ francés, pintura\}$$

Entonces, el $\vec{doc_1}$ quedará:

$$\begin{bmatrix}1 \\ 0 \\ 0 \\ 0 \\ 0\\ 0\end{bmatrix} +
  \begin{bmatrix}0 \\ 1 \\ 0 \\ 0 \\ 0\\ 0\end{bmatrix} =
  \begin{bmatrix}1 \\ 1 \\ 0 \\ 0 \\ 0\\ 0\end{bmatrix}$$

El $\vec{doc_2}$ quedará:

$$\begin{bmatrix}0 \\ 0 \\ 1 \\ 0 \\ 0\\ 0\end{bmatrix} +
  \begin{bmatrix}0 \\ 0 \\ 0 \\ 1 \\ 0\\ 0\end{bmatrix} +
  \begin{bmatrix}0 \\ 0 \\ 0 \\ 0 \\ 1\\ 0\end{bmatrix} =
  \begin{bmatrix}0 \\ 0 \\ 1 \\ 1 \\ 1\\ 0\end{bmatrix}$$

Y el $\vec{doc_3}$:

$$\begin{bmatrix}1 \\ 0 \\ 0 \\ 0 \\ 0\\ 0\end{bmatrix} +
  \begin{bmatrix}0 \\ 0 \\ 0 \\ 0 \\ 0\\ 1\end{bmatrix} =
  \begin{bmatrix}1 \\ 0 \\ 0 \\ 0 \\ 0\\ 1\end{bmatrix}$$



**¿Cuál es el problema?**

`buenísima` $\begin{bmatrix}1 \\ 0 \\ 0 \\ 0 \\ 0 \\0\end{bmatrix}$ y `espectacular` $ \begin{bmatrix}0 \\ 0 \\ 0 \\ 1 \\ 0 \\ 0\end{bmatrix}$ representan ideas muy similares. Por otra parte, sabemos que `marraqueta` $\begin{bmatrix}0 \\ 1 \\ 0 \\ 0 \\ 0 \\0\end{bmatrix}$ y `pan francés` $\begin{bmatrix}0 \\ 0 \\ 0 \\ 0 \\ 1 \\0\end{bmatrix}$ se refieren al mismo objeto. Pero en este modelo, estos **son totalmente distintos**. Es decir, los vectores de las palabras que `buenísima` y `espectacular` son tan distintas como `marraqueta` y `pan francés`. Esto se debe a que cada palabra ocupa una dimensión distinta a las demás y son completamente independientes. Esto evidentemente, repercute en la calidad de los modelos que creamos a partir de nuestro Bag of Words.

![BoW](https://raw.githubusercontent.com/dccuchile/CC6205/master/tutorials/recursos/BoW-Problem.png)



Ahora, si queremos ver que documento es mas similar a otro usando distancia euclidiana, veremos que:

$$d(doc_1, doc_2) = 2.236$$
$$d(doc_1, doc_3) = 1.414$$

Es decir, $doc_1$ se parece mas a $doc_3$ aunque nosotros sabemos que $doc_1$ y $doc_2$ nos están diciendo lo mismo!


Nos gustaría que eso no sucediera. Que existiera algún método que nos permitiera hacer que palabras similares tengan representaciones similares. Y que con estas, representemos mejor a los documentos, sin asumir que en el espacio son geométricamente equidistantes, ya que esto no es verdad en la vida real.


--------------------

### **Word Embeddings**

Es una de las representaciones más populares del vocabulario de un corpus. La idea principal de los Word Embeddings es crear representaciones vectoriales densas y de baja dimensionalidad $(d << |V|)$ de las palabras a partir de su contexto.

Volvamos a nuestro ejemplo anterior: `buenísima` y `espectacular` ocurren muchas veces en el mismo contexto, por lo que los embeddings que los representan debiesen ser muy similares... (*ejemplos de mentira hechos a mano*):

`buenísima` $\begin{bmatrix}0.32 \\ 0.44 \\ 0.92 \\ .001 \end{bmatrix}$ y `espectacular` $\begin{bmatrix}0.30 \\ 0.50 \\ 0.92 \\ .002 \end{bmatrix}$ versus `marraqueta`  $\begin{bmatrix}0.77 \\ 0.99 \\ 0.004 \\ .1 \end{bmatrix}$ el cuál es claramente distinto.


Pero, ¿Cuál es la utilidad de de crear estos vectores en NLP o en el área de Machine Learning en general?

Supongamos que tienen una enfermedad grave y deben ser operados el día de mañana. Le dan a elegir entre ser operados por un estudiante de primer año de medicina con algo de conocimiento médico o bien ser operados por un niño de 5 años 👶. ¿A quién elegirías?

Espero que tu opción haya sido el estudiante con una pequeña noción de los términos médicos implicados en una intervención así. Algo así es lo que se quiso lograr en el [paper](https://arxiv.org/abs/1301.3781) presentado por Mikolov en 2013, aludiendo a la herramienta **Word2Vec**. La idea es que si quieres resolver por ejemplo una tarea de clasificación de texto, ¿no sería útil utilizar el conocimiento de algún modelo pre-entrenado en una tarea similar de texto?. Claro, sería útil partir con los pesos entrenados por otra red, realizando lo que se llama **transfer learning**.

Ya pero.. ¿Cómo generamos estos vectores? ¿Cómo podemos capturar el contexto? ¿Cuál sería esa task auxiliar a utilizar?



##### **Word2vec y Skip-gram**

Word2Vec es probablemente el paquete de software mas famoso para crear word embeddings utilizando distintos modelos que emplean redes neuronales *shallow* o poco profundas.

Este nos provee herramientas para crear distintos tipos de modelos, tales como `Skip-Gram` y `Continuous Bag of Word (CBOW)`. En este caso, solo veremos `Skip-Gram`.

**Skip-gram** es una task auxiliar con la que crearemos nuestros embeddings. Esta tarea involucra tanto a las palabras y al contexto de ellas. Consiste en que por cada palabra del dataset, debemos predecir las palabras de su contexto (las palabras presentes en ventana de algún tamaño $k$).

![Overview](https://raw.githubusercontent.com/dccuchile/CC6205/master/tutorials/recursos/overview-skipgram.png)

Para resolverla, usaremos una red de una sola capa oculta. Los pesos ya entrenados de esta capa serán los que usaremos como embeddings.

#### Detalles del Modelo

- Como dijimos, el modelo será una red de una sola capa. La capa oculta tendrá una dimensión $d$ la cual nosotros determinaremos. Esta capa no tendrá función de activación. Sin embargo, la de salida si, la cual será una softmax para obtener las distribuciones de probabilidades y así ver cuáles palabras pertenecen o no al contexto.

- El vector de entrada, de tamaño $|V|$, será un vector one-hot de la palabra que estemos viendo en ese momento.

- La salida, también de tamaño $|V|$, será un vector que contenga la distribución de probabilidad de que cada palabra del vocabulario pertenezca al contexto de la palabra de entrada.

- Al entrenar, se comparará la distribución de los contextos con la suma de los vectores one-hot del contexto real.


(marraqueta, Estuvo), (marraqueta, buenisima), (marraqueta, la)
![Skip Gram](https://raw.githubusercontent.com/dccuchile/CC6205/master/tutorials/recursos/Skip-gram.png)


Nota: Esto es computacionalmente una locura. Por cada palabra de entrada, debemos calcular la probabilidad de aparición de todas las otras. Imaginen el caso de un vocabulario de 100.000 de palabras y de 10000000 oraciones...

La solución a esto es modificar la task a *Negative Sampling*. Esta transforma este problema de $|V|$ clases a uno binario.

### La capa Oculta y los Embeddings

Al terminar el entrenamiento, ¿Qué nos queda en la capa oculta?

Una matriz de $v$ filas por $d$ columnas, la cual contiene lo que buscabamos: Una representación continua de todas las palabras de nuestro vocabulario.  

**Cada fila de la matriz es un vector que contiene la representación continua una palabra del vocabulario.**


<img src="http://mccormickml.com/assets/word2vec/word2vec_weight_matrix_lookup_table.png" alt="Capa Oculta 1" style="width: 400px;"/>

¿Cómo la usamos eficientemente?

Simple: usamos los mismos vectores one-hot de la entrada y las multiplicamos por la matriz:

<img src="http://mccormickml.com/assets/word2vec/matrix_mult_w_one_hot.png" alt="Skip Gram" style="width: 400px;"/>

### Visualización

Veamos cómo se ven los embeddings de Word2Vec entrenados sobre un corpus gigante en Inglés. Para facilitar el análisis se reducen las 200 dimensiones a 3. El link a la visualización es el siguiente: Visualización: https://projector.tensorflow.org/

### Espacio multidimensional

Teniendo nuestro embeddings entonces podríamos hacer operaciones tan interesantes como las siguientes:

Manzana + Púrpura -> Ciruela

Rey - Hombre + Mujer -> Reina

Si bien no es posible obtener exactamente dichos vectores, esperaríamos que las palabras más cercanas al vector resultante serían las entregadas, obteniendo así un significado de las palabras según su contexto.


### Fuentes

Word2vec:
- mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/
- https://towardsdatascience.com/introduction-to-word-embedding-and-word2vec-652d0c2060fa

Gensim:
- https://www.kaggle.com/pierremegret/gensim-word2vec-tutorial

Nota: Las últimas 2 imagenes pertenecen a [Chris McCormick](http://mccormickml.com/about/)


## **Repaso: Entrenar nuestros Embeddings**

Para entrenar nuestros embeddings, usaremos el paquete gensim. Este trae una muy buena implementación de `word2vec`.




In [2]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 62.9 MB/s eta 0:00:00


In [28]:
import re
import pandas as pd
from time import time
from collections import defaultdict
import string
import multiprocessing
import os
import requests
import numpy as np

# word2vec
from gensim.models import Word2Vec, KeyedVectors
from gensim.models.phrases import Phrases, Phraser

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

# scikit-learn
import sklearn
from sklearn.manifold import TSNE
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.decomposition import PCA
from sklearn.base import BaseEstimator, TransformerMixin

# visualizaciones
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from ipywidgets import widgets

### Cargar el dataset y limpiar

Utilizaremos un datos de diversas fuentes de noticias que cuenta con tres atributos principales:

* texto: Referente al texto de la noticia.
* medio: Referente al medio de la noticia.
* fecha: Referente a la fecha de publicación.

In [4]:
dataset = pd.read_csv('https://raw.githubusercontent.com/giturra/JCC2023-WE/main/noticias_oct_dic_2019.tsv', sep='\t')

In [5]:
dataset.head()

,texto,medio,fecha
0,Rey de Tailandia despoja de títulos a su conso...,El Mercurio,2019-10-21 14:51:00
1,Denuncian que personas con epilepsia sufrieron...,El Mercurio,2019-12-18 12:12:00
2,Índice victimización de hogares en el país suf...,El Mercurio,2019-10-15 09:58:00
3,"Meditación, cocina, jardinería, manualidades: ...",El Mercurio,2019-11-16 16:42:00
4,Director de empresa que procesa datos electora...,El Mercurio,2019-11-01 05:25:00


In [6]:
# unir titulo con contenido de la noticia
content = dataset['texto']

In [7]:
content.head()

,texto
0,Rey de Tailandia despoja de títulos a su conso...
1,Denuncian que personas con epilepsia sufrieron...
2,Índice victimización de hogares en el país suf...
3,"Meditación, cocina, jardinería, manualidades: ..."
4,Director de empresa que procesa datos electora...


In [8]:
content.shape

(10000,)

In [9]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [10]:
from collections import Counter

# limpiar puntuaciones y separar por tokens.
punctuation = string.punctuation + "«»“”‘’…—"
stopwords = pd.read_csv(
    'https://raw.githubusercontent.com/Alir3z4/stop-words/master/spanish.txt'
).values
stopwords = Counter(stopwords.flatten().tolist())

def simple_tokenizer(doc, lower=False):
    if lower:
        tokenized_doc = doc.translate(str.maketrans(
            '', '', punctuation)).lower().split()

    tokenized_doc = doc.translate(str.maketrans('', '', punctuation)).split()
    tokenized_doc = [
        token for token in tokenized_doc if token.lower() not in stopwords
    ]
    return tokenized_doc

sentences = [simple_tokenizer(doc) for doc in content.values]

In [11]:
print("Ejemplo de alguna noticia: {}".format(sentences[14]))

Ejemplo de alguna noticia: ['Ley', 'Ciudadanía', 'polémica', 'norma', 'india', 'acusan', 'discriminatoria', 'provocado', 'oleada', 'protestas', 'semana', 'miles', 'personas', 'salido', 'a', 'manifestarse', 'semana', 'consecutiva', 'ciudades', 'India', 'mostrar', 'rechazo', 'Ley', 'Ciudadanía', 'CAB', 'migrantes', 'auspiciada', 'Gobierno', 'jornadas', 'terminado', 'enfrentamiento', 'marchas', 'vuelto', 'violentas', 'registran', '15', 'muertes', 'relacionadas', 'hechos', 'Pero¿por', 'rechazo', 'amplio', 'a', 'ley', 'gubernamental', 'ley', 'Ciudadanía', 'viene', 'a', 'reemplazar', 'a', 'norma', 'vigente', '64', 'años', 'prohíbe', 'migrantes', 'ilegales', 'pasen', 'a', 'ciudadanos', 'India', 'BBC', 'antigua', 'legislación', 'ilegales', 'a', 'inmigrantes', 'ingresan', 'país', 'pasaporte', 'vigente', 'documento', 'viaje', 'a', 'quedan', 'India', 'permitido', 'personas', 'deportadas', 'detenidas', 'ley', 'aprobó', '11', 'diciembre', 'anula', 'normativa', 'señala', 'persona', 'vivir', 'Gobiern

### Definir el modelo



Primero, como es usual, creamos el modelo. En este caso, usaremos uno de los primero modelos de embeddings neuronales: `word2vec`

Algunos parámetros importantes:

- `sentences`: Corresponde a una lista con la data para entrenar el modelo `word2vec`. En este caso a la lista tokenizada con el texto de las noticias.
- `min_count`: Ignora todas las palabras que tengan frecuencia menor a la indicada.
- `window` : Tamaño de la ventana. Usaremos 4.
- `size` : El tamaño de los embeddings que crearemos. Por lo general, el rendimiento sube cuando se usan mas dimensiones, pero después de 300 ya no se nota cambio. Ahora, usaremos solo 200.
- `workers`: Cantidad de CPU que serán utilizadas en el entrenamiento.

In [12]:
w2v = Word2Vec(sentences= sentences,
               min_count=10,
                      window=4,
                      vector_size=200,
                      sample=6e-5,
                      alpha=0.03,
                      min_alpha=0.0007,
                      negative=20,
                      workers=multiprocessing.cpu_count())

### Entrenar el modelo

Al definir el modelo de `Word2Vec` se construyó el vocabulario y se entrenó el modelo para obtener los `word embeddings` de los textos. Sin embargo estos 2 pasos también se pueden realizar de forma explícita para obtener un mayor control de los hiperparámetros.

#### Construir el vocabulario

Para esto, se creará un conjunto que contendrá (una sola vez) todas aquellas palabras que aparecen mas de `min_count` veces.

In [13]:
# w2v.build_vocab(sentences, progress_per=10000)

#### Entrenamiento

A continuación, entenaremos el modelo.
Los parámetros que usaremos serán:

- `total_examples`: Número de documentos.
- `epochs`: Número de veces que se iterará sobre el corpus.

Es recomendable que tengan instalado `cpython` antes de continuar. Aumenta bastante la velocidad de entrenamiento.


In [14]:
#t = time()
#w2v.train(sentences, total_examples=w2v.corpus_count, epochs=5, report_delay=10)
#print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

Ahora que terminamos de entrenar el modelo, le indicamos que no lo entrenaremos mas.
Esto nos permitirá ejecutar eficientemente las tareas que realizaremos.

In [15]:
w2v.init_sims(replace=True)

/tmp/ipython-input-2512947105.py:1: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  w2v.init_sims(replace=True)


####  Guardar y cargar el modelo

A su vez, para ahorrar tiempo, se pueden guardar y cargar modelos preentrenados.

In [16]:
"""
# Si entrenaste el modelo y lo quieres guardar, descomentar el siguiente bloque.
if not os.path.exists('./pretrained_models'):
    os.mkdir('./pretrained_models')
w2v.save('./pretrained_models/biobio_w2v.model')


# cargar el modelo (si es que lo entrenaron desde local.)
w2v = KeyedVectors.load("./pretrained_models/biobio_w2v.model", mmap='r')
"""

'\n# Si entrenaste el modelo y lo quieres guardar, descomentar el siguiente bloque.\nif not os.path.exists(\'./pretrained_models\'):\n    os.mkdir(\'./pretrained_models\')\nw2v.save(\'./pretrained_models/biobio_w2v.model\')\n\n\n# cargar el modelo (si es que lo entrenaron desde local.)\nw2v = KeyedVectors.load("./pretrained_models/biobio_w2v.model", mmap=\'r\')\n'

In [17]:
"""
# descargar el modelo desde github
def read_model_from_github(url):
    if not os.path.exists('./pretrained_models'):
        os.mkdir('./pretrained_models')

    r = requests.get(url)
    filename = url.split('/')[-1]
    with open('./pretrained_models/' + filename, 'wb') as f:
        f.write(r.content)
    return True


[
    read_model_from_github(file) for file in [
        'https://github.com/dccuchile/CC6205/releases/download/Data/biobio_w2v.model',
    ]
]
# cargar el modelo (si es que lo entrenaron desde local.)
w2v = KeyedVectors.load("./pretrained_models/biobio_w2v.model", mmap='r')
"""

'\n# descargar el modelo desde github\ndef read_model_from_github(url):\n    if not os.path.exists(\'./pretrained_models\'):\n        os.mkdir(\'./pretrained_models\')\n\n    r = requests.get(url)\n    filename = url.split(\'/\')[-1]\n    with open(\'./pretrained_models/\' + filename, \'wb\') as f:\n        f.write(r.content)\n    return True\n\n\n[\n    read_model_from_github(file) for file in [\n        \'https://github.com/dccuchile/CC6205/releases/download/Data/biobio_w2v.model\',\n    ]\n]\n# cargar el modelo (si es que lo entrenaron desde local.)\nw2v = KeyedVectors.load("./pretrained_models/biobio_w2v.model", mmap=\'r\')\n'

## **Word Embeddings como características para clasificar**


En esta sección, veremos como utilizar los word embeddings como característica para **clasificar noticias de diferentes medios**.


In [18]:
def to_vector(tokens,model):
    """ Receives a sentence string along with a word embedding model and
    returns the vector representation of the sentence"""
    vec = np.zeros(model.vectors.shape[1]) # creates an empty vector of 300 dimensions
    for word in tokens: # iterates over the sentence
        if word in model: # checks if the word is both in the word embedding and the tf-idf model
            vec += model[word] # adds every word embedding to the vector
    if np.linalg.norm(vec) > 0:
        return vec / np.linalg.norm(vec) # divides the vector by their normal
    else:
        return vec

In [19]:
dataset['text_vector'] = dataset['texto'].apply(lambda x: to_vector(simple_tokenizer(x, lower=True), w2v.wv))
display(dataset.head())

,texto,medio,fecha,text_vector
0,Rey de Tailandia despoja de títulos a su conso...,El Mercurio,2019-10-21 14:51:00,"[0.12115625310623257, 0.07681214341510867, 0.0..."
1,Denuncian que personas con epilepsia sufrieron...,El Mercurio,2019-12-18 12:12:00,"[0.08605537242728246, 0.03853898140059818, 0.0..."
2,Índice victimización de hogares en el país suf...,El Mercurio,2019-10-15 09:58:00,"[0.10995108521931063, 0.0353733183089699, 0.08..."
3,"Meditación, cocina, jardinería, manualidades: ...",El Mercurio,2019-11-16 16:42:00,"[0.10471409251158333, 0.05582559056423561, 0.0..."
4,Director de empresa que procesa datos electora...,El Mercurio,2019-11-01 05:25:00,"[0.08550749295611317, 0.06813306088471427, 0.0..."


### Dividir el dataset en training y test

In [35]:
X_train, X_test, y_train, y_test = train_test_split(dataset.text_vector,
                                                    dataset.medio,
                                                    test_size=0.33,
                                                    random_state=42)

# Convert the Series of vectors into a NumPy array
X_train = np.array(X_train.tolist())
X_test = np.array(X_test.tolist())
Y_train = np.array(y_train.tolist())
Y_test = np.array(y_test.tolist())

### Entrenamiento del clasificador

In [33]:
classifier = sklearn.linear_model.LogisticRegression(solver='liblinear')

In [34]:
classifier.fit(
    X_train,
    y_train
)

LogisticRegression(solver='liblinear')

In [39]:
y_pred = classifier.predict(X_test)

### Matriz de confusión

Como en cualquier problema de Machine Learning, la matriz de confusión es una herramienta fundamental para evaluar el rendimiento de un modelo de clasificación. Nos muestra un resumen del número de predicciones correctas e incorrectas, desglosadas por cada clase.

In [40]:
conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)

[[ 75  20  17  37  50  23  14   3  55  21]
 [  8 240  20   7  15  11   6   2  12  14]
 [ 12  64  72  28  67  38  11   6  41   6]
 [ 14  42   8 135  91  10   4   1  17   1]
 [ 15  20  30  73 180  20   2   5   4   2]
 [ 21  36  55  33  52  56  22   5  31   8]
 [ 17  11  15  23  25  11  99   6  72  28]
 [ 16  85  19  17  36  25  30  56  38  10]
 [ 23  14  14  16  17  12  12   2 183  27]
 [ 16  19  24  15  25  10  20   3  48 173]]


In [41]:
print(classification_report(
    y_test,
    y_pred
))

                 precision    recall  f1-score   support

            CHV       0.35      0.24      0.28       315
 Cooperativa CL       0.44      0.72      0.54       335
    El Mercurio       0.26      0.21      0.23       345
  El Rancaguino       0.35      0.42      0.38       323
   La Discusión       0.32      0.51      0.40       351
      La Nación       0.26      0.18      0.21       319
         La RED       0.45      0.32      0.38       307
           MEGA       0.63      0.17      0.27       332
Radio Concierto       0.37      0.57      0.45       320
     The Clinic       0.60      0.49      0.54       353

       accuracy                           0.38      3300
      macro avg       0.40      0.38      0.37      3300
   weighted avg       0.40      0.38      0.37      3300



### Clasificación con  Support Vector Machine (SVM)

In [42]:
from sklearn.svm import SVC

svm_classifier = SVC()
svm_classifier.fit(X_train, y_train)
y_pred_svm = svm_classifier.predict(X_test)

print("Confusion Matrix for SVM:")
print(confusion_matrix(y_test, y_pred_svm))

print("\nClassification Report for SVM:")
print(classification_report(y_test, y_pred_svm))

Confusion Matrix for SVM:
[[ 91  19  24  33  40  41  15   2  40  10]
 [ 10 234  21   9  13  16   5   2  12  13]
 [ 16  57  79  24  63  59  22   1  20   4]
 [ 17  26   6 118 105  21  16   1  12   1]
 [ 17  18  31  53 186  25  14   4   1   2]
 [ 30  29  58  28  44  79  20   1  24   6]
 [ 18   9  13  20  19  20 146   0  53   9]
 [ 21  75  19  15  29  43  13  75  33   9]
 [ 43   6  11  10  12  24  14   2 175  23]
 [ 38  16  23  16  20  25  19   1  29 166]]

Classification Report for SVM:
                 precision    recall  f1-score   support

            CHV       0.30      0.29      0.30       315
 Cooperativa CL       0.48      0.70      0.57       335
    El Mercurio       0.28      0.23      0.25       345
  El Rancaguino       0.36      0.37      0.36       323
   La Discusión       0.35      0.53      0.42       351
      La Nación       0.22      0.25      0.24       319
         La RED       0.51      0.48      0.49       307
           MEGA       0.84      0.23      0.36       3

### Clasificación Random Forest

In [43]:
from sklearn.ensemble import RandomForestClassifier

rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train, y_train)
y_pred_rf = rf_classifier.predict(X_test)

print("Confusion Matrix for Random Forest:")
print(confusion_matrix(y_test, y_pred_rf))

print("\nClassification Report for Random Forest:")
print(classification_report(y_test, y_pred_rf))

Confusion Matrix for Random Forest:
[[110  13  27  29  21  34   8  12  36  25]
 [ 19 230  24   7   5  15   6   8  10  11]
 [ 18  30 106  26  42  49  26  13  24  11]
 [ 13   9   7 178  61  24  20   0   9   2]
 [ 16   6  33  83 156  33   7   4   4   9]
 [ 38  14  65  24  32  78  17  13  24  14]
 [ 12   5  14  13  13  13 177   7  40  13]
 [ 17  33  16   8   8  22  12 183  23  10]
 [ 38   7   8  12  13  10  20   5 170  37]
 [ 46  10  21   8  14  22  11   4  25 192]]

Classification Report for Random Forest:
                 precision    recall  f1-score   support

            CHV       0.34      0.35      0.34       315
 Cooperativa CL       0.64      0.69      0.66       335
    El Mercurio       0.33      0.31      0.32       345
  El Rancaguino       0.46      0.55      0.50       323
   La Discusión       0.43      0.44      0.44       351
      La Nación       0.26      0.24      0.25       319
         La RED       0.58      0.58      0.58       307
           MEGA       0.73      0.

## Utilizando un pipeline

También, todo lo realizado anteriormente se puede definir en un pipeline.

Primero, crearemos el Transformer con el cual convertiremos el documento a vector.


### Doc2vec

In [22]:
class Doc2VecTransformer(BaseEstimator, TransformerMixin):
    """
    Transforma Noticias a representaciones vectoriales usando algún modelo de Word Embeddings.
    Recibe un modelo de Word Embeddings y una función de agregación. Esta última será necesaria
    para posteriormente representar el documento para un problema a resolver
    (como en este caso de clasificación).
    """

    def __init__(self, model, aggregation_func):
        # extraemos los embeddings desde el objeto contenedor. ojo con esta parte.
        self.model = model.wv

        # indicamos la función de agregación (np.min, np.max, np.mean, np.sum, ...)
        self.aggregation_func = aggregation_func

    def simple_tokenizer(self, doc, lower=False):
        """Tokenizador. Elimina signos de puntuación, lleva las letras a minúscula(opcional) y
           separa el Noticias por espacios.
        """
        if lower:
            doc.translate(str.maketrans('', '', string.punctuation)).lower().split()
        return doc.translate(str.maketrans('', '', string.punctuation)).split()

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):

        doc_embeddings = []

        for doc in X:
            # tokenizamos el documento. Se llevan todos los tokens a minúscula.
            # ojo con esto, ya que puede que tokens con minúscula y mayúscula tengan
            # distintas representaciones
            tokens = self.simple_tokenizer(doc, lower = True)

            selected_wv = []
            for token in tokens:
                if token in self.model.index_to_key:
                    selected_wv.append(self.model[token])

            # si seleccionamos por lo menos un embedding para el tweet, lo agregamos y luego lo añadimos.
            if len(selected_wv) > 0:
                doc_embedding = self.aggregation_func(np.array(selected_wv), axis=0)
                doc_embeddings.append(doc_embedding)
            # si no, añadimos un vector de ceros que represente a ese documento.
            else:
                print('No pude encontrar ningún embedding en el tweet: {}. Agregando vector de ceros.'.format(doc))
                doc_embeddings.append(np.zeros(self.model.vector_size)) # la dimension del modelo

        return np.array(doc_embeddings)


### Definimos el pipeline


Usaremos la transformación que creamos antes mas una regresión logística.

In [23]:
clf = LogisticRegression(max_iter=10)

doc2vec_mean = Doc2VecTransformer(w2v, np.mean)
doc2vec_sum = Doc2VecTransformer(w2v, np.sum)
doc2vec_max = Doc2VecTransformer(w2v, np.max)


pipeline = Pipeline([('doc2vec', doc2vec_sum), ('clf', clf)])

In [ ]:
pipeline.fit(X_train, y_train)

**Predecimos y evaluamos:**

In [ ]:
y_pred = pipeline.predict(X_test)

In [ ]:
conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
pipeline.predict(
    [("Alguna noticia..")])

## **Usandolo con BoW**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dataset.texto,
                                                    dataset.medio,
                                                    test_size=0.33,
                                                    random_state=42)

In [ ]:
# Definimos el vectorizador para convertir el texto a BoW:
vectorizer = CountVectorizer(analyzer='word', ngram_range=(1, 2))

# Definimos el clasificador que usaremos.
clf_2 = LogisticRegression(max_iter=10000)

# Definimos el pipeline
pipeline_2 = Pipeline([('features',
                        FeatureUnion([('bow', CountVectorizer()),
                                      ('doc2vec', doc2vec_sum)])), ('clf', clf)])

In [ ]:
pipeline_2.fit(X_train, y_train)

In [ ]:
y_pred_2 = pipeline_2.predict(X_test)
conf_matrix = confusion_matrix(y_test, y_pred_2)
print(conf_matrix)

In [ ]:
print(classification_report(y_test, y_pred_2))